In [ ]:
import numpy as np
import random 
## b3hash

In [ ]:
def okbithash(val,mod=0,mod2=0,mod3=0):
    mod = f'|{mod}|{mod2}|{mod3}|'
    s = val + mod
    h = hash(s)
    return(h%2)

In [ ]:
byte_count = np.array([0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 4, 5, 5, 6, 5, 6, 6, 7, 5, 6, 6, 7, 6, 7, 7, 8],dtype=np.uint8)

bitmasks = list(range(10))
bitmasks[0] = 0b00000001
bitmasks[1] = 0b00000010
bitmasks[2] = 0b00000100
bitmasks[3] = 0b00001000
bitmasks[4] = 0b00010000
bitmasks[5] = 0b00100000
bitmasks[6] = 0b01000000
bitmasks[7] = 0b10000000

class gbits:

    def __init__(self,gb):
        self.gb = gb
        self.num_bits = int(len(gb)*8)

    def __next__(self):
        if self.iter_idx >= self.num_bits:
            raise StopIteration()
        out = self.gb._getbit(self.iter_idx)
        self.iter_idx += 1
        return(out)

    def __getitem__(self,bit_idx):
        out = self.gb._getbit(bit_idx)
        return(out)

    def __iter__(self):
        self.iter_idx = 0
        return(self)
    
    def __len__(self):
        return(self.num_bits)

    def __repr__(self):
        s = f'<gbits>'
        return(s)


class gbytes(np.ndarray):
    def __new__(cls,b):
        buf = np.frombuffer(b,dtype=np.uint8)
        qa = np.ndarray.__new__(cls,len(b),dtype=np.uint8)
        qa[:] = buf
        return(qa)
    
    def __repr__(self):
        s = 'g\''
        for b in self:
            s += '\\x{:02x}'.format(b)
        s += '\''
        return(s)

    def _getbit(self,bit_idx):
        byte_idx = bit_idx // 8
        bit_idx = bit_idx % 8
        byte = self[byte_idx]
        bit = byte & bitmasks[bit_idx]
        bit = np.bool8(bit)
        return(bit)


    @property
    def bits(self):
        return(gbits(self))

    def bit_count(self):
        count = 0
        for b in self:
            count += byte_count[b]
        return(count)

    @staticmethod
    def similarity(gb1, gb2):
        sim_bin = gbytes(np.bitwise_not( np.bitwise_xor( gb1,gb2 ) ))
        sim = sim_bin.bit_count()
        sim = int(sim)
        return(sim)

        

In [ ]:
def b3hash(seq,rounds=256):
    ''' a much bettter version '''
    out_array = np.ndarray(rounds,dtype=np.bool8)
    seq = ["<START>"] + list(seq) + ["<END>"]
    last = True
    bigram_count = dict()
    for round in range(rounds):
        sim_sum = 0
        for i in range(1,len(seq)):
            bigram = (seq[i-1],seq[i])
            bigram_count.setdefault(bigram,0)
            count = bigram_count[bigram]
            h1 = okbithash(seq[i-1],round,mod2=0,mod3=count)
            h2 = okbithash(seq[i],round,mod2=1,mod3=count)
            bigram_count[bigram] += 1
            x12 = h1 ^ h2
            if x12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)


In [579]:
not_one_one(np.bool8(1),np.bool8(1))

False

In [651]:
def one_one(bit1,bit2): return(bit1 & bit2) #
def one_zero(bit1,bit2): return(bit1 & (bit1 ^ bit2))
def zero_one(bit1,bit2): return(bit2 & (bit1 ^ bit2)) 
def zero_zero(bit1,bit2): return( ~(bit1 | bit2)) 
def not_one_one(bit1,bit2): return(~(bit1 & bit2)) 
def not_one_zero(bit1,bit2): return(~(bit1 & (bit1 ^ bit2))) #
def not_zero_one(bit1,bit2): return(~(bit2 & (bit1 ^ bit2)))#
def not_zero_zero(bit1,bit2): return( (bit1 | bit2)) #
bit_funcs = np.array([one_one, one_zero, zero_one, zero_zero, not_one_one, not_one_zero, not_zero_one, not_zero_zero])

rstate = random.getstate()
random.seed(31337)
bitwheel_size = 256
bitwheel1 = np.array([random.randint(0,0xff) for _ in range(bitwheel_size)],dtype=np.uint8)
bitwheel2 = np.array([random.randint(0,0xff) for _ in range(bitwheel_size)],dtype=np.uint8)
bitwheel3 = np.array([random.randint(0,0xff) for _ in range(bitwheel_size)],dtype=np.uint8)
bitwheel3.dtype = np.bool8
random.setstate(rstate)

def itty_bibithash(bit1, bit2, mod1=0, mod2=0, mod3=0):
    bit1 = np.bool8(bit1)
    bit2 = np.bool8(bit2)
    idx = 0xff
    idx = idx ^ bitwheel1[mod1%len(bitwheel1)]
    idx = idx ^ bitwheel2[mod2%len(bitwheel2)]
    idx = idx ^ bitwheel3[mod3%len(bitwheel3)]
    idx = idx%len(bit_funcs)
    func = bit_funcs[idx]
    out = func(bit1,bit2)
    return(out)


In [652]:
zz = np.array([0,1])
zz[np.uint8(np.bool8(2))]

1

In [838]:
def itty_b3hash(b,rounds=64):
    bits = gbytes(b).bits
    bigram_count = np.array([0,0,0,0])
    out_array = np.ndarray(rounds,dtype=np.bool8)
    last = True
    for round in range(rounds):
        sim_sum = 0
        for i in range(1,len(bits)):
            b1 = bits[i-1]
            b2 = bits[i]
            bigram_ref = (b2<<1)+b1
            count = bigram_count[bigram_ref]
            #h12 = itty_bibithash(b1,b2,mod1=round,mod2=0)
            #if h12:
            #    sim_sum += .12
            #else:
            #    sim_sum -= .12
            h12 = itty_bibithash(b1,b2,mod1=round,mod2=count)
            bigram_count[bigram_ref] += 1
            if h12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)

In [841]:
zz = itty_b3hash(b'\xff\xff\xff\xff\xff\xff')
zzz = itty_b3hash(b'\xff\xff\xff\xff\xff\xff')


In [842]:
gbytes.similarity(zz,zzz)/64

1.0

In [269]:
cats = b3hash('cat')
caats = b3hash('caats')
caaaats = b3hash('caaaats')
catscats = b3hash('catscats')
bcatscats = b3hash('bcatscats')
caattaats = b3hash('caattaats')


In [270]:
elephant = b3hash('elephant')
eleephant = b3hash('eleephant')

In [271]:
alice = b3hash('Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')
april = b3hash('April was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')
april2 = b3hash('April was finally tired of sitting by her brother by the tree, and of having nothing to do')
ana = b3hash('of sitting by her nothing to do the bank, and of having Ana was beginning sister on to get very tired ')
#ana = b3hash('Ana was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')


long_sentence = b3hash('This is an extremely long sentence which should result in similarity for some reason ok bye')
longsentence = b3hash('Thisisanextremelylongsentencewhichshouldresultinsimilarityforsomereasonokbyesomemorewordshere')

shadowsfollow = b3hash('shadowsfollow')

In [272]:
garbage = b3hash('qwertysjdfniabuioopkjhgfdfvbsdljfnjkasdnxcvbkxjcvdfkjnndmmbyasdkjfbkhbsdfkzxcvibekxcbvkasdljfnkjksbdf')


In [273]:
pow = b3hash('aaaaab')
wow = b3hash('aaaaac')

In [274]:
garbage2 = b3hash(str([random.randint(87,122) for _ in range(5)]))

In [275]:
gbytes.similarity(pow,wow) / 256

0.77734375

In [276]:
gbytes.similarity(alice,april) / 256

0.8984375

In [277]:
gbytes.similarity(alice,april2) / 256

0.6484375

In [278]:
gbytes.similarity(alice,ana) / 256

0.87109375

In [279]:
gbytes.similarity(april,garbage2) / 256

0.53515625

In [280]:
gbytes.similarity(long_sentence,april) / 256

0.5859375

In [281]:
gbytes.similarity(long_sentence,alice) / 256

0.6015625

In [282]:
gbytes.similarity(garbage,april) / 256

0.53125

In [283]:
gbytes.similarity(longsentence,long_sentence) / 256

0.68359375

In [284]:
gbytes.similarity(garbage,longsentence) / 256

0.48046875

In [285]:
gbytes.similarity(caattaats,shadowsfollow) / 256

0.515625

In [286]:
gbytes.similarity(garbage2,april) / 256

0.53515625

In [287]:
garbage2

g'\xda\x8b\x71\xda\x96\xb3\xe9\x4d\x81\x9c\x87\x1b\x45\xb6\xae\x67\x19\xf0\x51\x8d\x0a\x56\x5d\x1f\x88\x27\xf9\x87\x17\x9d\x91\xe0'